# Non-Linear Mathematical Modelling

## Creating/Visualizing the Network

In [ ]:
# %pip install numpy matplotlib networkx
import numpy as np

In [ ]:
# Code taken from MSCI 332 Tutorial 2
# https://github.com/ST3042/MSCI332F22

# Random network generator
from itertools import combinations, groupby
import matplotlib.pyplot as plt
import networkx as nx

np.random.seed(8953456)
NUMBER_OF_INTERSECTIONS = 8
RANGE_OF_CAR = 336
MIN_ARC_LENGTH = 80
MAX_ARC_LENGTH = 200


# original version from https://stackoverflow.com/a/61961881
def gnp_random_connected_graph(n, p):
    """
    Generates a random undirected graph, similarly to an Erdős-Rényi
    graph, but enforcing that the resulting graph is conneted
    """
    edges = combinations(range(n), 2)
    G = nx.Graph()
    for i in range(n):
        G.add_node(i)

    if p <= 0:
        return G
    if p >= 1:
        return nx.complete_graph(n, create_using=G)
    for _, node_edges in groupby(edges, key=lambda x: x[0]):
        node_edges = list(node_edges)
        index = np.random.randint(len(node_edges))
        random_edge = node_edges[index]
        G.add_edge(
            random_edge[0],
            random_edge[1],
            weight=np.random.randint(MIN_ARC_LENGTH, MAX_ARC_LENGTH),
        )
        for e in node_edges:
            if np.random.random() < p:
                G.add_edge(*e, weight=np.random.randint(MIN_ARC_LENGTH, MAX_ARC_LENGTH))
    return G


nodes = NUMBER_OF_INTERSECTIONS
probability = 0.25
G = gnp_random_connected_graph(nodes, probability)
labels = nx.get_edge_attributes(G, "weight")
pos = nx.spring_layout(G, k=5)
plt.figure(figsize=(11, 9))
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
nx.draw(G, pos, with_labels=True, node_color="lightblue", node_size=1000)
plt.show()


The **network** above will be used to determine the optimal locations for placements of EV chargers. The graph is a fully connected graph where every vertice has some path leading to another. Every node is an intersection. The arcs have a weight that falls in the range of a random number between **MIN_ARC_LENGTH** and **MAX_ARC_LENGTH**. The max arc length is much smaller than the **Range** of the car. This is realistic because in the real world, there are many intersections in the path. Usually the intersection is closer than the range of the car. This assumption also allows us to avoid placing the charger almost everywhere in the network. 

## Deterministic Model

In [8]:
# %pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Read license from secret file
import json

secrets = json.load(open("../secrets.json"))

# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam("WLSACCESSID", secrets["GUROBI_WLSACCESSID"])
e.setParam("WLSSECRET", secrets["GUROBI_WLSSECRET"])
e.setParam("LICENSEID", secrets["GUROBI_LICENSEID"])
e.start()


In [ ]:
import itertools

graph = nx.to_dict_of_dicts(G)
shortest_path = nx.dijkstra_path(G, 0, 2)

shortest_paths = nx.all_pairs_dijkstra_path(G)

arc_lengths = nx.get_edge_attributes(G, "weight")
arc_lengths.update({(k[1], k[0]): v for k, v in arc_lengths.items()})


def get_arcs_from_nodes(node_list):
    return [(node_list[i - 1], node_list[i]) for i in range(1, len(node_list))]


routes = []
ARC_ROWS = set()
for start_node in shortest_paths:
    for b in start_node[1].values():
        if len(b) > 1:
            arcs = get_arcs_from_nodes(b)
            arcs_w_lengths = {arc: arc_lengths[arc] for arc in arcs}
            arc_row = list(arcs_w_lengths.items())[0]
            ARC_ROWS.add(arc_row)
            routes.append({"nodes": tuple(b), "arcs": arcs_w_lengths})
print(len(ARC_ROWS))


In [ ]:
ARC_MATRIX = []
for i in range(NUMBER_OF_INTERSECTIONS):
    matrix_row = [0 for i in range(NUMBER_OF_INTERSECTIONS)]

    for row in ARC_ROWS:
        if row[0][0] == i:
            matrix_row[row[0][1]] = row[1]
    ARC_MATRIX.append(matrix_row)


In [ ]:
ROUTES = []
ROUTE_DISTANCES = []
ROUTE_ARCS = []
for j in routes:
    ROUTES.append((list(j["nodes"])))
    arc_lens = j["arcs"]

    ROUTE_ARCS.append(list(arc_lens.keys()))

    arc_sum = sum(arc_lens.values())
    remaining_dist = []
    remaining_dist.append(arc_sum)
    for i in arc_lens.values():
        arc_sum -= i
        remaining_dist.append(arc_sum)
    ROUTE_DISTANCES.append(tuple(remaining_dist))


In [ ]:
ROUTE_ARC_BINARY = []

matrix = np.zeros(
    (len(ROUTES), NUMBER_OF_INTERSECTIONS, NUMBER_OF_INTERSECTIONS), dtype=int
)

for k in range(len(ROUTES)):
    arc = ROUTE_ARCS[k]

    for i in range(len(arc)):
        f = int(arc[i][0])
        s = int(arc[i][1])
        matrix[k][f][s] = 1

ROUTE_ARC_BINARY = matrix


In [ ]:
# Sets

I = list(G.nodes)  # intersections in the network
A = ARC_MATRIX  # lengths of feasible arcs/links between intersections
R = ROUTES  # intersections/nodes of routes
L = ROUTE_DISTANCES  # length of arcs/links of routes
F = ROUTE_ARCS  # arcs/links of routes


In [ ]:
# Parameters

## Constant
# 90% of the Range of the car
TAU = int(0.9 * RANGE_OF_CAR)
# Big-M: 10^9
M = 10**9

## Indexed
# Length of link (i,j)
a = np.array(A)
# Distance from intersection till destination
f = np.array(L, dtype=object)
# Additional range needed to reach destination
b = np.array([max(0, f[k][0] - TAU) for k, route in enumerate(L)])


In [ ]:
# Create the model within the Gurobi environment
model = gp.Model("EV Charging Station Optimization Model", env=e)

# Decision Variables

# If charging station exists at a node = 1, else = 0
x = model.addVars(I, vtype=GRB.BINARY, name="x")
# Number of chargers at a node
y = model.addVars(I, lb=0.0, vtype=GRB.INTEGER, name="y")
# If EV is charged at a node on a route = 1, else = 0
p = model.addVars(I, range(len(R)), vtype=GRB.BINARY, name="p")
# Range added to EV by charging at a node on a route
q = model.addVars(I, range(len(R)), lb=0.0, vtype=GRB.CONTINUOUS, name="q")
# Remaining range when entering a node on a route
r = model.addVars(I, range(len(R)), lb=0.0, vtype=GRB.CONTINUOUS, name="r")


In [ ]:
# Constraints
[model.addConstr(y[i] <= M * x[i]) for i in I]  # 1
[model.addConstr(y[i] >= x[i]) for i in I]  # 2
[model.addConstr(y[i] <= 10) for i in I]  # max number of chargers at station
[
    model.addConstr(M * x[i] >= quicksum(p[i, k] for k, route in enumerate(R)))
    for i in I
]  # 3
[
    [model.addConstr(q[i, k] <= M * p[i, k]) for i in route]
    for k, route in enumerate(R)
]  # 4
[[model.addConstr(q[i, k] >= p[i, k]) for i in route] for k, route in enumerate(R)]  # 5
[
    [model.addConstr(r[i, k] + q[i, k] <= TAU) for i in route]
    for k, route in enumerate(R)
]  # 6
[
    [model.addConstr(r[i, k] + q[i, k] == r[j, k] + a[i, j]) for (i, j) in arc]
    for k, arc in enumerate(F)
]  # 7
[
    model.addConstr(y[i] >= quicksum(p[i, k] for k, route in enumerate(R))) for i in I
]  # 8
[
    model.addConstr(quicksum(q[i, k] for i in I) == b[k]) for k, route in enumerate(R)
]  # 9
[
    model.addConstr(quicksum(p[i, k] for i in I) <= (b[k] / TAU) + 1)
    for k, route in enumerate(R)
]  # 10


In [ ]:
# Piecewise cost - non-linear objective function

# Cost Function
def chargers_cost(num_chargers):
    if num_chargers <= 0:
        return 0
    elif num_chargers <= 3:
        return 200_000 + 10_000 * num_chargers
    elif num_chargers <= 6:
        return 200_000 + 8_000 * num_chargers
    else:  # num_chargers > 6
        return 200_000 + 6_000 * num_chargers


# https://medium.com/bcggamma/hands-on-modeling-non-linearity-in-linear-optimization-problems-f9da34c23c9a
# SOS2 Variables

# SOS2 Breakpoints
d = [[0, 3, 6, 10]] * len(I)
# SOS2 Variable Definition
v = [model.addVars(len(d[i]), lb=0, vtype=GRB.CONTINUOUS) for i in range(len(I))]
[model.addSOS(GRB.SOS_TYPE2, v[i]) for i in range(len(I))]
# SOS2 Convexity Constraint
[model.addConstr(quicksum(v[i]) == 1) for i in range(len(I))]
# SOS2 Variable Constraint
[
    model.addConstr(quicksum([v[i][n] * d[i][n] for n in range(len(d[i]))]) == y[i])
    for i in range(len(I))
]

# Objective Function
model.setObjective(
    sum(sum(chargers_cost(d[i][n]) * v[i][n] for n in range(len(d[i]))) for i in I),
    sense=GRB.MINIMIZE,
)


In [ ]:
# Optimize
model.optimize()
print(f"NUMBER OF CONSTRAINTS: {len(model.getConstrs())}")


In [ ]:
# Output
try:
    print(f"\nOptimal cost: ${model.getObjective().getValue():,.0f}")
    color_map = []
    optimal_intersections = []
    for i in range(len(x)):
        # for j in y:
        if x[i].X and y[i].X:  # if variable is non-zero
            print(f"{int(y[i].X)} chargers will be place at intersection {i}")
            optimal_intersections.append(i)

    for node in G:
        if node in optimal_intersections:
            color_map.append("green")
        else:
            color_map.append("blue")

    # Draw Network
    plt.figure(figsize=(11, 9))
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    nx.draw(G, pos, with_labels=True, alpha=0.5, node_color=color_map, node_size=1000)
    plt.show()

except:
    print("Objective Solution Not Found")


As per the network graph above:

1. The green nodes are Charging Stations
2. The purple nodes are Intersections/Nodes